In [ ]:
pip install datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=93fab41baa49785e2a8fd02a0c009aa469fd3710fcaf46cc278c92a14437211b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attem

In [ ]:
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import datasets
from transformers import RobertaTokenizerFast, DefaultDataCollator, TrainingArguments, Trainer, AutoModelForQuestionAnswering
from datasets import Dataset
import json
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

def update_answer_starts_and_filter(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    updated_data = {"version": data.get("version", ""), "data": []}

    for article in data['data']:
        updated_article = {"title": article.get("title", ""), "paragraphs": []}

        for paragraph in article['paragraphs']:
            context = paragraph['context']
            updated_paragraph = {"context": context, "qas": []}

            for qa in paragraph['qas']:
                updated_qa = {"question": qa['question'], "id": qa['id'], "answers": []}

                for answer in qa['answers']:
                    answer_text = answer['text']
                    answer_start = context.find(answer_text)
                    if answer_start != -1:
                        updated_qa['answers'].append({"text": answer_text, "answer_start": answer_start})

                if updated_qa['answers']:
                    updated_paragraph['qas'].append(updated_qa)

            if updated_paragraph['qas']:
                updated_article['paragraphs'].append(updated_paragraph)

        if updated_article['paragraphs']:
            updated_data['data'].append(updated_article)

    return updated_data

def save_updated_data(data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

input_file_path = '/content/drive/MyDrive/model_giaoduc/datagd.json'
output_file_path = '/content/drive/MyDrive/model_giaoduc/updateDataGd.json'

updated_data = update_answer_starts_and_filter(input_file_path)
save_updated_data(updated_data, output_file_path)


In [ ]:
import json
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file JSON
with open('/content/drive/MyDrive/model_giaoduc/updateDataGd.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

contexts = []
questions = []
answers = []
for article in data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answer = qa['answers'][0]['text'] if qa['answers'] else None
            answer_start = context.find(answer) if answer else None
            if answer:
                contexts.append(context)
                questions.append(question)
                answers.append({
                    "text": [answer],
                    "start": [answer_start]
                })

# Chuyển dữ liệu thành định dạng list of tuples
data = list(zip(contexts, questions, answers))

# Chia dữ liệu thành train và val (80% train, 20% temp)
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)



# Chuyển đổi dữ liệu trở lại thành các dictionary để lưu vào file JSON
def convert_to_dict(data):
    contexts, questions, answers = zip(*data)
    return {
        'context': list(contexts),
        'question': list(questions),
        'answer': list(answers)
    }

train_dict = convert_to_dict(train_data)
val_dict = convert_to_dict(val_data)

# Lưu dữ liệu vào các file JSON
with open('train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_dict, f, ensure_ascii=False, indent=4)

with open('val_data.json', 'w', encoding='utf-8') as f:
    json.dump(val_dict, f, ensure_ascii=False, indent=4)



In [ ]:
import json
from sklearn.model_selection import train_test_split

def load_and_preprocess_squad(input_file, tokenizer, max_length=258):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    contexts = []
    questions = []
    answers = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text'] if qa['answers'] else None
                answer_start = qa['answers'][0]['answer_start'] if qa['answers'] else None
                if answer:
                    # Xử lý trường hợp context lớn hơn max_length
                    if len(tokenizer.encode(context)) > max_length:
                        start_pos = max(0, answer_start - max_length // 2)
                        end_pos = min(len(context), start_pos + max_length)
                        context = context[start_pos:end_pos]
                        answer_start = context.find(answer)

                    contexts.append(context)
                    questions.append(question)
                    if not isinstance(answer_start, (int, float)):
                        answer_start = 0
                    answers.append({
                        "text": [answer.lower()],
                        "start": [answer_start]
                    })

    # Kiểm tra độ dài của các cột
    assert len(contexts) == len(questions) == len(answers)

    # Tạo từ điển dữ liệu
    dataset = {
        'context': contexts,
        'question': questions,
        'answer': answers
    }

    return dataset

def split_data(dataset, test_size=0.1):
    contexts = dataset['context']
    questions = dataset['question']
    answers = dataset['answer']

    train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(
        contexts, questions, answers, test_size=test_size, random_state=42
    )

    train_dataset = {
        'context': train_contexts,
        'question': train_questions,
        'answer': train_answers
    }

    val_dataset = {
        'context': val_contexts,
        'question': val_questions,
        'answer': val_answers
    }

    return train_dataset, val_dataset




In [ ]:
input_file = '/content/drive/MyDrive/model_giaoduc/updateDataGd.json'

tokenizer = RobertaTokenizerFast.from_pretrained('vinai/phobert-base')
model = AutoModelForQuestionAnswering.from_pretrained("vinai/phobert-base")
model.train()

dataset = load_and_preprocess_squad(input_file, tokenizer)

train_dataset, val_dataset, test_dataset = split_data(dataset, test_size=0.1)

print(f"Train dataset size: {len(train_dataset['context'])}")
print(f"Validation dataset size: {len(val_dataset['context'])}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train dataset size: 500
Validation dataset size: 56


In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=256,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["start"][0]
        end_char = answer["start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
dataset_train = Dataset.from_dict(train_dataset)
dataset_eval = Dataset.from_dict(val_dataset)

In [ ]:
tokenized_squad = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)
tokenized_squad_eval = dataset_eval.map(preprocess_function, batched=True, remove_columns=dataset_eval.column_names)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

# Sử dụng DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, return_tensors='pt')

# Cấu hình các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="phobert_law",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad,
    eval_dataset=tokenized_squad_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    )

# Huấn luyện mô hình
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.952300
2,No log,0.624322
3,No log,0.461667
4,No log,0.393678
5,No log,0.383627
6,No log,0.396402
7,No log,0.374264
8,No log,0.427243
9,No log,0.413757
10,No log,0.475958


TrainOutput(global_step=960, training_loss=0.3100018600622813, metrics={'train_runtime': 686.4843, 'train_samples_per_second': 21.85, 'train_steps_per_second': 1.398, 'total_flos': 1959725675520000.0, 'train_loss': 0.3100018600622813, 'epoch': 30.0})

In [ ]:
path = "/content/drive/MyDrive/modelgiaoduc"

trainer.save_model(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/modelgiaoduc/tokenizer_config.json',
 '/content/drive/MyDrive/modelgiaoduc/special_tokens_map.json',
 '/content/drive/MyDrive/modelgiaoduc/vocab.json',
 '/content/drive/MyDrive/modelgiaoduc/merges.txt',
 '/content/drive/MyDrive/modelgiaoduc/added_tokens.json',
 '/content/drive/MyDrive/modelgiaoduc/tokenizer.json')

# Test Model

In [ ]:
input_file = '/content/drive/MyDrive/model_giaoduc/updateDataGd.json'

dataset = load_and_preprocess_squad(input_file, tokenizer)

train_dataset, val_dataset = split_data(dataset, test_size=0.1)

In [ ]:
model_save_path = '/content/drive/MyDrive/model_giaoduc'
tokenizer = RobertaTokenizerFast.from_pretrained(model_save_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_save_path)

In [65]:
def check_question_length(question, max_length=258):
    if len(question) > max_length:
        return True
    return False

In [70]:
def answer_question(question, context, model, tokenizer):
    if check_question_length(question):
        return "câu hỏi quá dài!"
    inputs = tokenizer(question.lower(), context, return_tensors="pt",max_length=258, padding="max_length", truncation="only_second")
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    print("Context: " + context + "\n")
    print("Question " + question+ "\n")

    if(tokenizer.decode(predict_answer_tokens) == ""):
        print("Answer: Chưa thể tìm thấy câu trả lời \n")
        answer_result = "Chưa thể tìm thấy câu trả lời"
    else:
        print("Answer: " + tokenizer.decode(predict_answer_tokens)+ "\n")
        answer_result = tokenizer.decode(predict_answer_tokens)
    return answer_result

In [67]:
ques="Em là sinh viên khóa 05DH do bảo lưu nên hiện tại em học cùng khóa với 06DH nhưng lớp em vẫn được giữ lớp cũ nên vẫn là 05DH trong đợt đăng kí môn học vừa qua vì Trường xếp ngày đầu tiên chỉ cho khóa 06DH đăng kí nên em bị đẩy lùi xuống đăng kí sau 2 ngày mà ngày đăng kí đó tất cả các lớp thuộc khóa 06DH đều bị khóa hết không cho đăng kí em phải gọi lên xin mở để em được đăng kí nhưng khi vào được thì hầu hết các lớp đã đủ số lượng vì là năm 4 nên rất khó để có thể đủ 30 bạn lập thành danh sách xin mở lớp,em mong trong đợt đăng kí ở học kì sau Trường có thể cho khóa 05DH được đăng kí cùng đợt với khóa 06DH vì số lượng bảo lưu như em rất ít nếu đăng kí lùi như đợt này nữa chúng em sẽ gặp nhiều khó khăn trong thực tập, đồ án em mong Thầy (Cô) xem xét điều chỉnh giúp em. Em cảm ơn."
contx="chào em; do phân luồng theo hệ để đăng ký môn học không bị nghẽn; nếu em gặp trục trặc môn nào trong đợt đăng ký có thể lên phòng đào tạo để cô hỗ trợ"
answer_question(ques,contx, model, tokenizer)

'câu hỏi quá dài!'

In [72]:
from collections import Counter
import string
import re
def normalize_answer(s):
    def white_space_fix(text):
        return ' '.join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(lower(s))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(pred_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def evaluate_model(val_dataset, model, tokenizer):
    f1 = 0
    em = 0
    total = len(val_dataset['context'])

    for i in range(total):
      context = val_dataset['context'][i]
      question = val_dataset['question'][i]
      true_answer = val_dataset['answer'][i]['text'][0]

      predicted_answer = answer_question(question, context, model, tokenizer)
      f1 += f1_score(predicted_answer, true_answer)
      em += exact_match_score(predicted_answer, true_answer)

    f1 = f1 / total
    em = em / total
    return f1, em

In [73]:
f1, em = evaluate_model(val_dataset, model, tokenizer)
print(f"Validation F1 Score: {f1 * 100:.2f}%")
print(f"Validation Exact Match Score: {em * 100:.2f}%")

Context: Phòng Đào tạo – tầng trệt nhà C

Question vì gia đình em khó khăn, em muốn xin tạm dừng kết quả đang học thì liên hệ ở phòng nào ạ?

Answer: Phòng Đào tạo – tầng trệt nhà C

Context: chào em; em muốn được biết chuyển dc hay không; thì em liên hệ bên trường Ngoại ngữ xem họ có nhận ko nhé.

Question Thưa thầy/cô, em hiện là sinh viên năm nhất chuyên ngành công nghệ thông tin, em xin hỏi là em có được xét duyệt để chuyển trường đến đại học ngoại ngữ tin học (cùng chuyên ngành công nghệ thông tin) được không ạ? Em xin cảm ơn, mong thầy/cô hồi đáp giúp em.

Answer: chào em; em muốn được biết chuyển dc hay không; thì em liên hệ bên trường Ngoại ngữ xem họ có nhận ko nhé.

Context: Người học không thuộc diện bị buộc thôi học, thực hiện đầy đủ nghĩa vụ và trách nhiệm theo quy định, nếu thuộc một trong các trường hợp sau được đề nghị trường cho thôi học: Người học tự xét thấy không còn khả năng hoàn thành chương trình do thời gian còn lại không đủ để hoàn thành chương trình theo quy đ

In [ ]:
# Đưa model về thiết bị (CPU hoặc GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay